In [13]:
import pandas as pd 
import numpy as np
import regex as re

In [11]:
df = pd.read_csv("Outlets_Cleaned.csv", index_col = [0] )

#### Handling NA Values

In [17]:
#Sometimes there is no join date, therefore rather than removing these entries we take the average join date and input that
round(df.describe(),0)
df.loc[df['Join'].isna() == True, "Join"] = df['Join'].median()

In [18]:
df = df.dropna().reset_index(drop=True)

In [3]:
#Rows wich have the same rumour by the same outlet
df = df.drop_duplicates(subset=['Outlet','Year','Player','Interested_clubs'], keep='first')

#### Converting the positions to Attack, Midfield, Defence

In [6]:
Attackers = ["Left Winger", "Centre-Forward", "Right Winger", "Second Striker"]
Midfielders = ['Attacking Midfield', 'Central Midfield','Left Midfield', 'Defensive Midfield','Right Midfield']
Defenders = ['Left-Back','Right-Back', 'Centre-Back', 'Goalkeeper']

In [7]:
df.loc[df["Position"].isin(Attackers) , "Position"] = "Attacker"
df.loc[df["Position"].isin(Midfielders) , "Position"] = "Midfielder"
df.loc[df["Position"].isin(Defenders) , "Position"] = "Defender"

#### Adding Player Agents

In [8]:
from bs4 import BeautifulSoup
import requests

In [9]:
headers = {
    # 'Host' : 'https://www.zim.com/',
    'Connection': 'keep-alive',
    'User-Agent': 'Chrome/102.0.5005.63 Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.5 (KHTML, like Gecko) Safari/536.5',
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate',
    'Content-Type': 'text',
    'Accept-Language': 'en-US,en;q=0.8'
}

def get_data(URL):
    soup = BeautifulSoup(requests.get(URL, headers=headers).content, 'lxml')
    agent = 'Unknown'
    try:
        agent = soup.find('span', {'onclick': 'tmEvent("spielerprofil", "click", "berater-spielerdaten")'}).text.replace('\n', '')
    except:
        pass
    
    try:
        agent = soup.find('a', {'onclick': 'tmEvent("spielerprofil", "click", "berater-spielerdaten")'}).text.replace('\n', '')
    except:
        pass
    
    return agent

In [10]:
#In case a line causes problems
#df = df.drop(df.index[395])

In [11]:
#agent = []
#n = 0
#for url in df["Player Link"] : 
#    agent.append(get_data(url))
#    n += 1        

In [11]:
df.insert(3, "Agent", agent)

In [231]:
df.to_csv('Agents_Data.csv', encoding='utf-8')

#### Data for Outlet Analysis

In [137]:
df_outlet = pd.read_csv('Agents_Data.csv', index_col = [0] )
df_outlet = df_outlet.drop_duplicates(subset=['Outlet','Year','Player'], keep='first')
df_outlet = df_outlet.drop(['Rumour', 'Year', 'Player', 'Club', 'Interested_clubs', 'Clubs', 'Player Link'], axis=1)

In [138]:
df_outlet = df_outlet.iloc[:,[0,2,8]]

In [139]:
df_outlet["Outlet"].value_counts()[0:14]

Mail                       146
The Sun                    123
Sky Sports                  93
The Athletic                92
Football Insider            73
Mirror                      72
90 Min                      64
Marca                       63
Calcio mercato              62
L'Equipe                    58
Fabrizio Romano             55
Corriere dello Sport        53
Independent Journalists     45
Goal                        42
Name: Outlet, dtype: int64

In [140]:
df_outlet = df_outlet.groupby('Outlet').filter(lambda x : len(x)>70)

In [141]:
#features = pd.get_dummies(df_outlet)

In [142]:
df_outlet.to_csv('Outlet_Analysis_Data.csv', encoding='utf-8')

#### Data for Regression

In [114]:
df_reg = pd.read_csv('Agents_Data.csv', index_col = [0] )

In [115]:
df_reg = df_reg.drop_duplicates(subset=['Outlet','Year','Player'], keep='first')

In [116]:
len(df_reg)

1638

In [117]:
n = 14
df_reg = df_reg.groupby('League').filter(lambda x : len(x)>n)
df_reg = df_reg.groupby('Country').filter(lambda x : len(x)>n)
df_reg = df_reg.groupby('Outlet').filter(lambda x : len(x)>n)
df_reg = df_reg.groupby('Agent').filter(lambda x : len(x)>7)

In [118]:
#Storing for later
continuous = df_reg[["Market Value", "Join", "Age"]]

In [119]:
df_reg = df_reg.iloc[:,[2,3,6,8,10,15]]
#df_reg = df_reg.drop(['Rumour', 'Year', 'Player', 'Club', 'Interested_clubs', "Market Value", "Join", "Age"], axis=1)

In [120]:
features = pd.get_dummies(df_reg)

In [121]:
df_reg = pd.concat([continuous, features], axis=1)

In [83]:
df_reg.to_csv('Regression_Data.csv', encoding='utf-8')

#### Categorical Variables Thresholds

In [4]:
df_analysis = pd.read_csv('Agents_Data.csv', index_col = [0] )

In [5]:
df_analysis = df_analysis.drop_duplicates(subset=['Outlet','Year','Player'], keep='first')

In [6]:
df_analysis = df_analysis.drop(['Rumour', 'Year', 'Player', 'Club', 'Interested_clubs', 'Clubs', 'Player Link'], axis=1)

In [7]:
#Checking lengths of categorical variables
print(len(df_analysis["League"].value_counts()))
print(len(df_analysis["Country"].value_counts()))
print(len(df_analysis["Outlet"].value_counts()))
print(len(df_analysis["Agent"].value_counts()))
print(len(df_analysis))

11
60
139
253
1638


In [8]:
n = 14

In [9]:
df_analysis = df_analysis.groupby('League').filter(lambda x : len(x)>n)
df_analysis = df_analysis.groupby('Country').filter(lambda x : len(x)>n)
df_analysis = df_analysis.groupby('Outlet').filter(lambda x : len(x)>n)
df_analysis = df_analysis.groupby('Agent').filter(lambda x : len(x)>7)

In [10]:
df_analysis["Outlet"].value_counts()

Mail                       83
The Sun                    66
The Athletic               51
Sky Sports                 47
Football Insider           44
90 Min                     41
Marca                      40
Mirror                     39
L'Equipe                   30
Telegraph                  28
Independent Journalists    26
Fabrizio Romano            26
Goal                       24
Mundo Deportivo            23
Star                       21
Calcio mercato             19
Times                      17
Bild                       17
Corriere dello Sport       17
Talksport                  16
Evening Standard           16
Express                    15
Sport                      15
Guardian                   15
ESPN                       14
Metro                      13
Manchester Evening News    13
Liverpool Echo              9
Name: Outlet, dtype: int64

In [24]:
#Shortening Names
df_analysis.loc[df_analysis['Outlet'].str.contains("Independent Journalists"), "Outlet"] = "Journalists"
df_analysis.loc[df_analysis['Outlet'].str.contains("Fabrizio Romano"), "Outlet"] = "F. Romano" 
df_analysis.loc[df_analysis['Outlet'].str.contains("Manchester Evening News"), "Outlet"] = "Manchester N." 
df_analysis.loc[df_analysis['Outlet'].str.contains("Evening Standard"), "Outlet"] = "Evening Std" 
df_analysis.loc[df_analysis['Outlet'].str.contains("Liverpool Echo"), "Outlet"] = "Liverpool E." 
df_analysis.loc[df_analysis['Outlet'].str.contains("Corriere dello Sport"), "Outlet"] = "Corriere S." 
df_analysis.loc[df_analysis['Outlet'].str.contains("Football Insider"), "Outlet"] = "Insider" 
df_analysis.loc[df_analysis['Outlet'].str.contains("Calcio mercato"), "Outlet"] = "Calcio M." 
df_analysis.loc[df_analysis['Outlet'].str.contains("Mundo Deportivo"), "Outlet"] = "Mundo D." 
df_analysis.loc[df_analysis['Outlet'].str.contains("The Athletic"), "Outlet"] = "The Athletic"

In [25]:
#Checking lengths of categorical variables after thresholds
print(len(df_analysis["League"].value_counts()))
print(len(df_analysis["Country"].value_counts()))
print(len(df_analysis["Outlet"].value_counts()))
print(len(df_analysis["Agent"].value_counts()))
print(len(df_analysis))

10
24
28
29
785


In [26]:
#df1 = df_analysis.loc[(df_analysis['Outlet'].str.contains("F. Romano") & df_analysis['Outcome'] == True), ]
#df2 = df_analysis.loc[(df_analysis['Outlet'].str.contains("Marca") & df_analysis['Outcome'] == True), ]
#
#df3 = df_analysis.loc[df_analysis['Outlet'].str.contains("Mail"), ]
#df4 = df_analysis.loc[df_analysis['Outlet'].str.contains("Sun"), ]
#
#df_analysis = pd.concat([ df1, df3, df3, df4], axis=0)

In [27]:
#Testing proportions of different categorical variables
xxx = df_analysis.loc[df_analysis['Outcome'] == True , ]
print(len(df_analysis.loc[df_analysis['Join'] > 2000,]))
print(len(xxx.loc[xxx['Join'] > 2000,]))

print(len(df_analysis.loc[df_analysis['Country'].str.contains("England") & df_analysis['Outcome'] == True , ]))
print(len(df_analysis.loc[df_analysis['Country'].str.contains("England"),]))


200
81
55
208


In [28]:
df_analysis.to_csv('Analysis_Data.csv', encoding='utf-8')

In [15]:
df_analysis.describe()

,Age,Join,Market Value
count,785.000000,785.000000,785.000000
mean,25.414013,1519.861146,33.259745
std,4.102396,1045.936904,27.230780
min,16.000000,-62.000000,0.000000
25%,23.000000,730.000000,14.000000
50%,25.000000,1096.000000,25.000000
75%,28.000000,2130.000000,45.000000
max,37.000000,6209.000000,160.000000
